<a href="https://colab.research.google.com/github/MdAbuRummanRefat/RealTimeEmotion-GenderClassification/blob/main/GenderClassificationTrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
data=pd.read_csv('/content/drive/My Drive/Project/imdb4p5.csv')
data.head(5)

,Unnamed: 0,gender,age,px0,px1,px2,px3,px4,px5,px6,px7,px8,px9,px10,px11,px12,px13,px14,px15,px16,px17,px18,px19,px20,px21,px22,px23,px24,px25,px26,px27,px28,px29,px30,px31,px32,px33,px34,px35,px36,...,px9960,px9961,px9962,px9963,px9964,px9965,px9966,px9967,px9968,px9969,px9970,px9971,px9972,px9973,px9974,px9975,px9976,px9977,px9978,px9979,px9980,px9981,px9982,px9983,px9984,px9985,px9986,px9987,px9988,px9989,px9990,px9991,px9992,px9993,px9994,px9995,px9996,px9997,px9998,px9999
0,0,0,49,16,15,17,18,15,15,15,16,19,16,17,20,18,19,19,22,23,22,22,23,21,21,21,20,26,26,29,36,30,28,32,51,69,80,64,50,50,...,22,111,56,207,184,48,15,11,11,13,81,11,9,10,10,10,10,10,10,9,10,13,22,16,13,16,114,82,7,7,11,23,193,132,11,5,12,12,14,14
1,1,0,81,47,48,49,47,49,51,51,54,54,55,53,52,56,54,55,47,45,38,38,44,53,59,60,59,60,61,62,57,49,41,38,39,40,41,37,37,42,...,140,46,39,32,29,36,31,29,31,29,30,31,32,33,31,29,30,31,31,31,33,33,35,33,30,31,31,31,31,32,33,33,33,33,33,33,33,33,33,32
2,2,0,81,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,21,19,19,19,21,22,20,22,21,22,25,22,22,23,23,24,24,24,23,23,23,...,12,11,8,8,8,11,11,14,11,12,11,11,10,10,10,12,13,6,6,5,6,6,9,9,10,9,6,7,8,8,8,8,8,7,6,5,8,4,227,217
3,3,0,80,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,39,52,43,49,46,47,61,58,44,61,65,50,43,49,50,52,52,46,54,54,50,47,53,48,47,47,45,49,43,47,36,59,39,19,9,27,25,24,24,24
4,4,1,37,184,191,199,201,195,197,198,208,213,215,216,221,220,222,220,213,204,163,150,178,204,215,224,229,231,230,230,230,230,230,230,229,230,230,231,231,229,...,162,173,174,174,168,175,168,171,169,179,174,161,168,161,165,229,244,240,233,223,217,196,175,188,225,230,234,232,232,230,230,228,226,226,224,221,217,217,211,205


In [4]:
labels = {0:'female', 1: 'male'}
emotion_counts = data['gender'].value_counts().reset_index()
emotion_counts.columns = ['gender', 'number']
emotion_counts['gender'] = emotion_counts['gender'].map(labels)
emotion_counts

,gender,number
0,female,18777
1,male,14370


In [5]:
dataset=data.values

In [8]:
X=dataset[:,3:]
y=dataset[:,1]

In [10]:
X=np.asarray(X)

In [12]:
from keras.utils import to_categorical
def pre_Processing(X,Y):
    X = X.reshape(-1,100,100,1)
    X = X/255.0
    Y = to_categorical(Y,2) 
    return X,Y
X_set,Y_set = pre_Processing(X,y)


In [22]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_set,Y_set,test_size=.2,random_state=42)
X_val,X_test,y_val,y_test=train_test_split(X_test,y_test,test_size=.5,random_state=41)

In [23]:
y_test.shape

(3315, 2)

In [24]:
from keras.layers import Activation, Convolution2D, Dropout, Conv2D
from keras.layers import AveragePooling2D, BatchNormalization
from keras.layers import GlobalAveragePooling2D
from keras.models import Sequential
from keras.layers import Flatten
from keras.models import Model
from keras.layers import Input
from keras.layers import MaxPooling2D
from keras.layers import SeparableConv2D
from keras import layers
from keras.regularizers import l2

In [25]:
def mini_XCEPTION(input_shape, num_classes, l2_regularization=0.01):
    regularization = l2(l2_regularization)

    # base
    img_input = Input(input_shape)
    x = Conv2D(8, (3, 3), strides=(1, 1), kernel_regularizer=regularization,
               use_bias=False)(img_input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(8, (3, 3), strides=(1, 1), kernel_regularizer=regularization,
               use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # module 1
    residual = Conv2D(16, (1, 1), strides=(2, 2),
                      padding='same', use_bias=False)(x)
    residual = BatchNormalization()(residual)

    x = SeparableConv2D(16, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(16, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
    x = BatchNormalization()(x)

    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = layers.add([x, residual])

    # module 2
    residual = Conv2D(32, (1, 1), strides=(2, 2),
                      padding='same', use_bias=False)(x)
    residual = BatchNormalization()(residual)

    x = SeparableConv2D(32, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(32, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
    x = BatchNormalization()(x)

    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = layers.add([x, residual])

    # module 3
    residual = Conv2D(64, (1, 1), strides=(2, 2),
                      padding='same', use_bias=False)(x)
    residual = BatchNormalization()(residual)

    x = SeparableConv2D(64, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(64, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
    x = BatchNormalization()(x)

    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = layers.add([x, residual])

    # module 4
    residual = Conv2D(128, (1, 1), strides=(2, 2),
                      padding='same', use_bias=False)(x)
    residual = BatchNormalization()(residual)

    x = SeparableConv2D(128, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(128, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
    x = BatchNormalization()(x)

    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = layers.add([x, residual])

    x = Conv2D(num_classes, (3, 3),
               # kernel_regularizer=regularization,
               padding='same')(x)
    x = GlobalAveragePooling2D()(x)
    output = Activation('softmax', name='predictions')(x)

    model = Model(img_input, output)
    return model

In [27]:
# parameters
batch_size = 32
num_epochs = 110
input_shape = (100, 100, 1)
verbose = 1
num_classes = 2
patience = 100

In [28]:
# model parameters/compilation
model = mini_XCEPTION(input_shape, num_classes)
model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100, 100, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 98, 98, 8)    72          input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 98, 98, 8)    32          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 98, 98, 8)    0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [29]:
from keras.preprocessing.image import ImageDataGenerator
# data generator
data_generator = ImageDataGenerator(
                        featurewise_center=False,
                        featurewise_std_normalization=False,
                        rotation_range=10,
                        width_shift_range=0.1,
                        height_shift_range=0.1,
                        zoom_range=.1,
                        horizontal_flip=True)

In [30]:

from keras.callbacks import CSVLogger, ModelCheckpoint, EarlyStopping
from keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau('val_loss', factor=0.1,
                                  patience=int(patience/4), verbose=1)

In [31]:
model.fit(data_generator.flow(X_train, y_train,batch_size),
                        steps_per_epoch=len(X_train) // batch_size,
                        epochs=num_epochs, verbose=1, callbacks=[reduce_lr],
                        validation_data=(X_val,y_val))

Epoch 1/110
828/828 [==============================] - 64s 77ms/step - loss: 0.4185 - accuracy: 0.8397 - val_loss: 0.2430 - val_accuracy: 0.9243
Epoch 2/110
828/828 [==============================] - 63s 76ms/step - loss: 0.2143 - accuracy: 0.9326 - val_loss: 0.2165 - val_accuracy: 0.9318
Epoch 3/110
828/828 [==============================] - 63s 76ms/step - loss: 0.1700 - accuracy: 0.9486 - val_loss: 0.3303 - val_accuracy: 0.8827
Epoch 4/110
828/828 [==============================] - 63s 76ms/step - loss: 0.1449 - accuracy: 0.9564 - val_loss: 0.7585 - val_accuracy: 0.7201
Epoch 5/110
828/828 [==============================] - 63s 77ms/step - loss: 0.1318 - accuracy: 0.9612 - val_loss: 0.1262 - val_accuracy: 0.9674
Epoch 6/110
828/828 [==============================] - 63s 76ms/step - loss: 0.1243 - accuracy: 0.9623 - val_loss: 0.1568 - val_accuracy: 0.9535
Epoch 7/110
828/828 [==============================] - 63s 76ms/step - loss: 0.1185 - accuracy: 0.9661 - val_loss: 0.1361 - val_ac

In [32]:
model.evaluate(X_test,y_test)

104/104 [==============================] - 1s 6ms/step - loss: 0.0769 - accuracy: 0.9825


[0.07688559591770172, 0.9825037717819214]

In [33]:
model.save('/content/drive/My Drive/Project/Gender_simple_CNN.81-0.96.hdf5')